In [41]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import cv2 as cv
import os

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier

import sys
sys.path.append('./Do_s_python_utils')
from resize_imgs import *
from flatten_imgs_np_array import *

## resizing segmented files. 

In [5]:
seg_benign_path = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented\\benign'
seg_malignant_path = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented\\malignant'
seg_benign_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28\\benign'
seg_malignant_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28\\malignant'
resize_imgs(seg_benign_path, seg_benign_path_28, (28,28))
resize_imgs(seg_malignant_path, seg_malignant_path_28, (28,28))

## X and y (flattened and splitted)

In [27]:
seg_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28'
X, y = flatten_imgs_np_array(seg_path_28)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

### Training logistic with L1 Penalty

In [54]:
lr_clf = LR(penalty = 'l1', solver = 'liblinear')
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))

[[20 17]
 [17 21]]


In [55]:
cross_val_score(lr_clf, X_train, y_train, cv = 10 )

array([0.625     , 0.79166667, 0.56521739, 0.72727273, 0.45454545,
       0.68181818, 0.68181818, 0.59090909, 0.59090909, 0.68181818])

### Training SVC